In [10]:
import pandas as pd
import re
import astropy.io.fits as pf
import numpy as np
import matplotlib.pyplot as plt
import astropy.io as ai
import desispec.io
import desispec.coaddition
import os

In [11]:
def extracting_spectrum(targetid, tileid, paddleid):
    directory = '/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/'+str(tileid)+'/'
    find_dir = os.listdir(directory)
    new_directory = directory+find_dir[0]+'/'
    print(new_directory)
    filename = 'coadd-'+str(paddleid)+'-'+str(TILEID[i_tile])+'-thru'+str(find_dir[0])+'.fits'
    coadd = desispec.io.read_spectra(new_directory+filename)
    combine = desispec.coaddition.coadd_cameras(coadd)
    index = []
    print(len(targetid))
    
    output2 = np.zeros((len(targetid),7781))
    output3 = np.zeros((len(targetid),7781))
    for i in range(0,len(targetid)):
        #index.append(np.where(np.array(coadd.fibermap['TARGETID'])==targetid[i])[0][0])
    
        if len(np.where(np.array(coadd.fibermap['TARGETID'])==targetid[i])[0])>0:
            
            index = np.where(np.array(coadd.fibermap['TARGETID'])==targetid[i])[0][0]
            #print(index)
            output1 = combine.wave['brz']
            output2[i,:]= combine.flux['brz'][index]
            output3[i,:]= combine.ivar['brz'][index]
        else:
            output1 = np.arange(0,7781)*0.-999
            output2[i,:]= np.arange(0,7781)*0.-999
            output3[i,:]= np.arange(0,7781)*0.-999  
  
    return output1,output2,output3
    #return combine.wave['brz'],combine.flux['brz'][np.array(index)],combine.ivar['brz'][np.array(index)]

In [12]:
catalog = pd.read_csv('Everest_vs_denali_dz_GT_00033.csv')
catalog = catalog[(catalog['DELTACHI2_everest']<100) & (catalog['DELTACHI2_everest']>20)]
all_tileid = catalog['TILEID_everest']
all_targetid = catalog['TARGETID']
FIBERID = catalog['FIBER_everest']
all_paddeid = FIBERID // 500

In [13]:
TILEID = list(set(np.array(all_tileid)))
paddleid = [0,1,2,3,4,5,6,7,8,9]
flux_matrix = np.zeros((len(all_targetid),7781,2))-999

In [14]:
for i_tile in range(0,len(TILEID)):
    for i_paddleid in paddleid:
        search = np.where((all_tileid==TILEID[i_tile]) & (all_paddeid==i_paddleid))
        tmp_targetid = np.array(all_targetid)[search[0]]
        if len(tmp_targetid)>0:
            directory = '/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/'+str(TILEID[i_tile])+'/'
            find_dir = os.listdir(directory)
            new_directory = directory+find_dir[0]+'/'
            print(new_directory)
            filename = 'coadd-'+str(i_paddleid)+'-'+str(TILEID[i_tile])+'-thru'+str(find_dir[0])+'.fits'
            #if (i_paddleid!=2) | (TILEID[i_tile]!=325):
            if os.path.isfile(new_directory+filename)==True:
                wave, flux_output,ivar_output = extracting_spectrum(tmp_targetid,TILEID[i_tile],i_paddleid)
                for i_search in range(0,len(tmp_targetid)):
                    flux_matrix[search[0][i_search],:,0]=flux_output[i_search]
                    flux_matrix[search[0][i_search],:,1]=ivar_output[i_search]

/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
INFO:spectra.py:282:read_spectra: iotime 0.818 sec to read coadd-0-80896-thru20210318.fits at 2021-09-02T00:19:15.880158
2
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
INFO:spectra.py:282:read_spectra: iotime 0.699 sec to read coadd-2-80896-thru20210318.fits at 2021-09-02T00:19:20.286119
2
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
INFO:spectra.py:282:read_spectra: iotime 0.773 sec to read coadd-5-80896-thru20210318.fits at 2021-09-02T00:19:24.391646
1
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative/80896/20210318/
INFO:spectra.py

In [15]:
catalog.to_csv('Everest_vs_denali_deltach2_lt_100_gt_20_Denali.csv')

In [16]:
hdu_spectra = pf.PrimaryHDU(flux_matrix)
hdu_wave = pf.ImageHDU(wave)
#hdu_catalog = pf.BinTableHDU(t2)
hdulist = pf.HDUList([hdu_spectra,hdu_wave])
hdulist.writeto('./Everest_vs_denali_deltachi2_lt_100_gt_20_spectra_Denali.fits',overwrite=True)